# Geração de Data Schemas - Projeto ECD

Este notebook coleta automaticamente os metadados e amostras de dados de todas as tabelas do projeto ECD.

## O que será gerado:

Para cada tabela:
- **DESCRIBE FORMATTED** - Metadados completos (schema, partições, localização, etc)
- **SELECT * FROM ... LIMIT 10** - Amostra de 10 linhas de dados

## Totais:
- **52 tabelas** (originais + intermediárias)
- **104 comandos SQL** (2 por tabela)
- **156 arquivos** de saída (DESCRIBE.txt, SAMPLE.txt, SAMPLE.csv)

---

## 1. Configuração Inicial

In [ ]:
import sys
import os
from datetime import datetime
from pathlib import Path
import warnings

# Adicionar paths necessários
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

from pyspark.sql.functions import *
from utils import spark_utils_session as utils

# Configurações
warnings.filterwarnings('ignore')

print("✓ Imports realizados com sucesso")

## 2. Inicializar Sessão Spark

In [ ]:
def get_session(profile: str = 'efd_t2', dynamic_allocation_enabled: bool = True):
    """Gera DBASparkAppSession."""
    
    app_name = "ecd_data_schema_generator"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

# Inicializar sessão
print("Inicializando sessão Spark...")
session = get_session(profile='efd_t2')
spark = session.sparkSession
print("✓ Sessão Spark inicializada com sucesso")

# Testar conexão
spark.sql("SHOW DATABASES").limit(5).show(truncate=False)

## 3. Definição das Tabelas

Organize as tabelas por categoria para facilitar o processamento.

In [ ]:
TABELAS = {
    "ORIGINAIS_RI": [
        "usr_sat_ecd.ecd_ri050_plano_contas",
        "usr_sat_ecd.ecd_ri051_plano_contas_referencial",
        "usr_sat_ecd.ecd_ri150_saldos_periodicos_identificacao_periodo",
        "usr_sat_ecd.ecd_ri155_detalhe_saldos_periodicos",
    ],
    
    "ORIGINAIS_RJ": [
        "usr_sat_ecd.ecd_rj100_balanco_patrimonial",
        "usr_sat_ecd.ecd_rj150_demonstracao_resultado_exercicio",
    ],
    
    "ORIGINAIS_PROCESSADAS": [
        "teste.ecd_i150",
        "teste.ecd_i200",
    ],
    
    "PRODUCAO": [
        "teste.ecd_contas_classificadas_producao",
        "teste.ecd_balanco_patrimonial",
        "teste.ecd_dre",
        "teste.ecd_indicadores_financeiros",
    ],
    
    "STREAMLIT": [
        "teste.ecd_empresas_cadastro",
        "teste.ecd_score_risco_consolidado",
        "teste.ecd_saldos_contas_v2",
        "teste.ecd_plano_contas",
    ],
    
    "ML_DATASET": [
        "teste.ecd_ml_dataset",
        "teste.ecd_ml_train",
        "teste.ecd_ml_val",
        "teste.ecd_ml_test",
    ],
    
    "ML_PREDICOES": [
        "teste.ecd_ml_predictions_ALL",
        "teste.ecd_ml_predictions_rf_val",
        "teste.ecd_ml_predictions_lr_val",
        "teste.ecd_ml_predicoes",
    ],
    
    "ML_METRICAS": [
        "teste.ecd_ml_metricas",
        "teste.ecd_ml_performance_por_classe",
        "teste.ecd_ml_erros_rf",
        "teste.ml_label_mapping",
    ],
    
    "ML_EMPRESAS": [
        "teste.ecd_ml_stats_classificacao_empresa",
        "teste.ecd_ml_valores_balanco_empresa",
        "teste.ecd_ml_valores_dre_empresa",
        "teste.ecd_ml_empresas_consolidado",
    ],
    
    "ML_ANALISE": [
        "teste.ecd_ml_empresas_aptas_indicadores",
        "teste.ecd_ml_empresas_indices_padrao",
        "teste.ecd_ml_indices_padrao_decis",
        "teste.ecd_ml_candidatas_ajuste_manual",
        "teste.ecd_ml_sinteticas_por_heranca",
        "teste.ecd_ml_fallback_classificacoes",
    ],
    
    "INDICADORES": [
        "teste.ecd_indicadores_hibrido",
        "teste.ecd_indices_padrao_setoriais",
        "teste.ecd_empresas_classificacao_resumo",
        "teste.ecd_evolucao_scores",
        "teste.ecd_empresas_criticas",
    ],
    
    "VALIDACAO": [
        "teste.pc_referencia_completa",
        "teste.ecd_contas_classificadas_final",
        "teste.ecd_resumo_executivo",
        "teste.ecd_detalhamento_metodo",
        "teste.ecd_top_classificacoes",
        "teste.ecd_empresas_equacao_ok",
        "teste.ecd_amostra_ml",
        "teste.ecd_contas_nao_classificadas",
        "teste.ecd_stats_por_empresa",
    ],
}

# Estatísticas
total_tabelas = sum(len(tabelas) for tabelas in TABELAS.values())
print(f"Total de categorias: {len(TABELAS)}")
print(f"Total de tabelas: {total_tabelas}")
print(f"Total de comandos SQL: {total_tabelas * 2}")
print("\nTabelas por categoria:")
for categoria, tabelas in TABELAS.items():
    print(f"  - {categoria}: {len(tabelas)} tabelas")

## 4. Funções Auxiliares

In [ ]:
def verificar_tabela_existe(tabela_nome):
    """Verifica se a tabela existe no banco."""
    try:
        spark.sql(f"DESCRIBE {tabela_nome}")
        return True
    except Exception as e:
        return False

def coletar_describe_formatted(tabela_nome):
    """Coleta DESCRIBE FORMATTED de uma tabela."""
    try:
        df = spark.sql(f"DESCRIBE FORMATTED {tabela_nome}")
        return df
    except Exception as e:
        print(f"  ✗ Erro ao coletar DESCRIBE FORMATTED: {str(e)}")
        return None

def coletar_sample_data(tabela_nome):
    """Coleta amostra de dados (10 linhas) de uma tabela."""
    try:
        df = spark.sql(f"SELECT * FROM {tabela_nome} LIMIT 10")
        return df
    except Exception as e:
        print(f"  ✗ Erro ao coletar sample: {str(e)}")
        return None

def salvar_dataframe_como_texto(df, arquivo_path):
    """Salva DataFrame como arquivo de texto."""
    try:
        pdf = df.toPandas()
        with open(arquivo_path, 'w', encoding='utf-8') as f:
            f.write(pdf.to_string(index=False))
        return True
    except Exception as e:
        print(f"  ✗ Erro ao salvar arquivo: {str(e)}")
        return False

def salvar_dataframe_como_csv(df, arquivo_path):
    """Salva DataFrame como CSV."""
    try:
        pdf = df.toPandas()
        pdf.to_csv(arquivo_path, index=False, encoding='utf-8')
        return True
    except Exception as e:
        print(f"  ✗ Erro ao salvar CSV: {str(e)}")
        return False

print("✓ Funções auxiliares definidas")

## 5. Configurar Diretório de Saída

In [ ]:
# Criar diretório de saída
output_dir = Path("/home/user/ECD/data-schemas")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Diretório de saída: {output_dir}")
print(f"✓ Diretório criado/verificado")

## 6. Processar Tabelas

### ⚙️ Escolha o que processar:

**Opção 1:** Execute todas as células abaixo para processar TODAS as categorias

**Opção 2:** Execute apenas as células das categorias que você precisa

---

### 6.1. ORIGINAIS_RI (4 tabelas)

In [ ]:
categoria = "ORIGINAIS_RI"
tabelas = TABELAS[categoria]

print(f"{'='*80}")
print(f"  📂 CATEGORIA: {categoria}")
print(f"  Total de tabelas: {len(tabelas)}")
print(f"{'='*80}\n")

categoria_dir = output_dir / categoria
categoria_dir.mkdir(parents=True, exist_ok=True)

for i, tabela in enumerate(tabelas, 1):
    print(f"\n[{i}/{len(tabelas)}] 📊 Processando: {tabela}")
    
    if not verificar_tabela_existe(tabela):
        print(f"  ⚠️  Tabela não encontrada")
        continue
    
    tabela_limpa = tabela.replace('.', '_')
    
    # DESCRIBE FORMATTED
    print(f"  → Coletando DESCRIBE FORMATTED...")
    df_describe = coletar_describe_formatted(tabela)
    if df_describe:
        arquivo_describe = categoria_dir / f"{tabela_limpa}_DESCRIBE.txt"
        if salvar_dataframe_como_texto(df_describe, arquivo_describe):
            print(f"  ✓ DESCRIBE salvo")
    
    # SELECT SAMPLE
    print(f"  → Coletando sample (10 linhas)...")
    df_sample = coletar_sample_data(tabela)
    if df_sample:
        arquivo_sample_txt = categoria_dir / f"{tabela_limpa}_SAMPLE.txt"
        arquivo_sample_csv = categoria_dir / f"{tabela_limpa}_SAMPLE.csv"
        
        if salvar_dataframe_como_texto(df_sample, arquivo_sample_txt):
            print(f"  ✓ Sample TXT salvo")
        if salvar_dataframe_como_csv(df_sample, arquivo_sample_csv):
            print(f"  ✓ Sample CSV salvo")

print(f"\n{'='*80}")
print(f"  ✅ Categoria {categoria} concluída!")
print(f"{'='*80}")

### 6.2. ORIGINAIS_RJ (2 tabelas)

In [ ]:
categoria = "ORIGINAIS_RJ"
tabelas = TABELAS[categoria]

print(f"{'='*80}")
print(f"  📂 CATEGORIA: {categoria}")
print(f"  Total de tabelas: {len(tabelas)}")
print(f"{'='*80}\n")

categoria_dir = output_dir / categoria
categoria_dir.mkdir(parents=True, exist_ok=True)

for i, tabela in enumerate(tabelas, 1):
    print(f"\n[{i}/{len(tabelas)}] 📊 Processando: {tabela}")
    
    if not verificar_tabela_existe(tabela):
        print(f"  ⚠️  Tabela não encontrada")
        continue
    
    tabela_limpa = tabela.replace('.', '_')
    
    # DESCRIBE FORMATTED
    print(f"  → Coletando DESCRIBE FORMATTED...")
    df_describe = coletar_describe_formatted(tabela)
    if df_describe:
        arquivo_describe = categoria_dir / f"{tabela_limpa}_DESCRIBE.txt"
        if salvar_dataframe_como_texto(df_describe, arquivo_describe):
            print(f"  ✓ DESCRIBE salvo")
    
    # SELECT SAMPLE
    print(f"  → Coletando sample (10 linhas)...")
    df_sample = coletar_sample_data(tabela)
    if df_sample:
        arquivo_sample_txt = categoria_dir / f"{tabela_limpa}_SAMPLE.txt"
        arquivo_sample_csv = categoria_dir / f"{tabela_limpa}_SAMPLE.csv"
        
        if salvar_dataframe_como_texto(df_sample, arquivo_sample_txt):
            print(f"  ✓ Sample TXT salvo")
        if salvar_dataframe_como_csv(df_sample, arquivo_sample_csv):
            print(f"  ✓ Sample CSV salvo")

print(f"\n{'='*80}")
print(f"  ✅ Categoria {categoria} concluída!")
print(f"{'='*80}")

### 6.3. ORIGINAIS_PROCESSADAS (2 tabelas)

In [ ]:
categoria = "ORIGINAIS_PROCESSADAS"
tabelas = TABELAS[categoria]

print(f"{'='*80}")
print(f"  📂 CATEGORIA: {categoria}")
print(f"  Total de tabelas: {len(tabelas)}")
print(f"{'='*80}\n")

categoria_dir = output_dir / categoria
categoria_dir.mkdir(parents=True, exist_ok=True)

for i, tabela in enumerate(tabelas, 1):
    print(f"\n[{i}/{len(tabelas)}] 📊 Processando: {tabela}")
    
    if not verificar_tabela_existe(tabela):
        print(f"  ⚠️  Tabela não encontrada")
        continue
    
    tabela_limpa = tabela.replace('.', '_')
    
    # DESCRIBE FORMATTED
    print(f"  → Coletando DESCRIBE FORMATTED...")
    df_describe = coletar_describe_formatted(tabela)
    if df_describe:
        arquivo_describe = categoria_dir / f"{tabela_limpa}_DESCRIBE.txt"
        if salvar_dataframe_como_texto(df_describe, arquivo_describe):
            print(f"  ✓ DESCRIBE salvo")
    
    # SELECT SAMPLE
    print(f"  → Coletando sample (10 linhas)...")
    df_sample = coletar_sample_data(tabela)
    if df_sample:
        arquivo_sample_txt = categoria_dir / f"{tabela_limpa}_SAMPLE.txt"
        arquivo_sample_csv = categoria_dir / f"{tabela_limpa}_SAMPLE.csv"
        
        if salvar_dataframe_como_texto(df_sample, arquivo_sample_txt):
            print(f"  ✓ Sample TXT salvo")
        if salvar_dataframe_como_csv(df_sample, arquivo_sample_csv):
            print(f"  ✓ Sample CSV salvo")

print(f"\n{'='*80}")
print(f"  ✅ Categoria {categoria} concluída!")
print(f"{'='*80}")

### 6.4. PRODUCAO (4 tabelas) ⭐ PRIORITÁRIO

In [ ]:
categoria = "PRODUCAO"
tabelas = TABELAS[categoria]

print(f"{'='*80}")
print(f"  📂 CATEGORIA: {categoria}")
print(f"  Total de tabelas: {len(tabelas)}")
print(f"{'='*80}\n")

categoria_dir = output_dir / categoria
categoria_dir.mkdir(parents=True, exist_ok=True)

for i, tabela in enumerate(tabelas, 1):
    print(f"\n[{i}/{len(tabelas)}] 📊 Processando: {tabela}")
    
    if not verificar_tabela_existe(tabela):
        print(f"  ⚠️  Tabela não encontrada")
        continue
    
    tabela_limpa = tabela.replace('.', '_')
    
    # DESCRIBE FORMATTED
    print(f"  → Coletando DESCRIBE FORMATTED...")
    df_describe = coletar_describe_formatted(tabela)
    if df_describe:
        arquivo_describe = categoria_dir / f"{tabela_limpa}_DESCRIBE.txt"
        if salvar_dataframe_como_texto(df_describe, arquivo_describe):
            print(f"  ✓ DESCRIBE salvo")
    
    # SELECT SAMPLE
    print(f"  → Coletando sample (10 linhas)...")
    df_sample = coletar_sample_data(tabela)
    if df_sample:
        arquivo_sample_txt = categoria_dir / f"{tabela_limpa}_SAMPLE.txt"
        arquivo_sample_csv = categoria_dir / f"{tabela_limpa}_SAMPLE.csv"
        
        if salvar_dataframe_como_texto(df_sample, arquivo_sample_txt):
            print(f"  ✓ Sample TXT salvo")
        if salvar_dataframe_como_csv(df_sample, arquivo_sample_csv):
            print(f"  ✓ Sample CSV salvo")

print(f"\n{'='*80}")
print(f"  ✅ Categoria {categoria} concluída!")
print(f"{'='*80}")

### 6.5. STREAMLIT (4 tabelas)

In [ ]:
categoria = "STREAMLIT"
tabelas = TABELAS[categoria]

print(f"{'='*80}")
print(f"  📂 CATEGORIA: {categoria}")
print(f"  Total de tabelas: {len(tabelas)}")
print(f"{'='*80}\n")

categoria_dir = output_dir / categoria
categoria_dir.mkdir(parents=True, exist_ok=True)

for i, tabela in enumerate(tabelas, 1):
    print(f"\n[{i}/{len(tabelas)}] 📊 Processando: {tabela}")
    
    if not verificar_tabela_existe(tabela):
        print(f"  ⚠️  Tabela não encontrada")
        continue
    
    tabela_limpa = tabela.replace('.', '_')
    
    # DESCRIBE FORMATTED
    print(f"  → Coletando DESCRIBE FORMATTED...")
    df_describe = coletar_describe_formatted(tabela)
    if df_describe:
        arquivo_describe = categoria_dir / f"{tabela_limpa}_DESCRIBE.txt"
        if salvar_dataframe_como_texto(df_describe, arquivo_describe):
            print(f"  ✓ DESCRIBE salvo")
    
    # SELECT SAMPLE
    print(f"  → Coletando sample (10 linhas)...")
    df_sample = coletar_sample_data(tabela)
    if df_sample:
        arquivo_sample_txt = categoria_dir / f"{tabela_limpa}_SAMPLE.txt"
        arquivo_sample_csv = categoria_dir / f"{tabela_limpa}_SAMPLE.csv"
        
        if salvar_dataframe_como_texto(df_sample, arquivo_sample_txt):
            print(f"  ✓ Sample TXT salvo")
        if salvar_dataframe_como_csv(df_sample, arquivo_sample_csv):
            print(f"  ✓ Sample CSV salvo")

print(f"\n{'='*80}")
print(f"  ✅ Categoria {categoria} concluída!")
print(f"{'='*80}")

### 6.6. ML_DATASET (4 tabelas)

In [ ]:
categoria = "ML_DATASET"
tabelas = TABELAS[categoria]

print(f"{'='*80}")
print(f"  📂 CATEGORIA: {categoria}")
print(f"  Total de tabelas: {len(tabelas)}")
print(f"{'='*80}\n")

categoria_dir = output_dir / categoria
categoria_dir.mkdir(parents=True, exist_ok=True)

for i, tabela in enumerate(tabelas, 1):
    print(f"\n[{i}/{len(tabelas)}] 📊 Processando: {tabela}")
    
    if not verificar_tabela_existe(tabela):
        print(f"  ⚠️  Tabela não encontrada")
        continue
    
    tabela_limpa = tabela.replace('.', '_')
    
    # DESCRIBE FORMATTED
    print(f"  → Coletando DESCRIBE FORMATTED...")
    df_describe = coletar_describe_formatted(tabela)
    if df_describe:
        arquivo_describe = categoria_dir / f"{tabela_limpa}_DESCRIBE.txt"
        if salvar_dataframe_como_texto(df_describe, arquivo_describe):
            print(f"  ✓ DESCRIBE salvo")
    
    # SELECT SAMPLE
    print(f"  → Coletando sample (10 linhas)...")
    df_sample = coletar_sample_data(tabela)
    if df_sample:
        arquivo_sample_txt = categoria_dir / f"{tabela_limpa}_SAMPLE.txt"
        arquivo_sample_csv = categoria_dir / f"{tabela_limpa}_SAMPLE.csv"
        
        if salvar_dataframe_como_texto(df_sample, arquivo_sample_txt):
            print(f"  ✓ Sample TXT salvo")
        if salvar_dataframe_como_csv(df_sample, arquivo_sample_csv):
            print(f"  ✓ Sample CSV salvo")

print(f"\n{'='*80}")
print(f"  ✅ Categoria {categoria} concluída!")
print(f"{'='*80}")

### 6.7. Processar Categorias Restantes (AUTOMATIZADO)

Esta célula processa todas as categorias ML restantes de uma vez.

In [ ]:
# Categorias restantes para processar
categorias_restantes = [
    "ML_PREDICOES",
    "ML_METRICAS",
    "ML_EMPRESAS",
    "ML_ANALISE",
    "INDICADORES",
    "VALIDACAO"
]

for categoria in categorias_restantes:
    tabelas = TABELAS[categoria]
    
    print(f"\n{'='*80}")
    print(f"  📂 CATEGORIA: {categoria}")
    print(f"  Total de tabelas: {len(tabelas)}")
    print(f"{'='*80}\n")
    
    categoria_dir = output_dir / categoria
    categoria_dir.mkdir(parents=True, exist_ok=True)
    
    for i, tabela in enumerate(tabelas, 1):
        print(f"\n[{i}/{len(tabelas)}] 📊 Processando: {tabela}")
        
        if not verificar_tabela_existe(tabela):
            print(f"  ⚠️  Tabela não encontrada")
            continue
        
        tabela_limpa = tabela.replace('.', '_')
        
        # DESCRIBE FORMATTED
        print(f"  → Coletando DESCRIBE FORMATTED...")
        df_describe = coletar_describe_formatted(tabela)
        if df_describe:
            arquivo_describe = categoria_dir / f"{tabela_limpa}_DESCRIBE.txt"
            if salvar_dataframe_como_texto(df_describe, arquivo_describe):
                print(f"  ✓ DESCRIBE salvo")
        
        # SELECT SAMPLE
        print(f"  → Coletando sample (10 linhas)...")
        df_sample = coletar_sample_data(tabela)
        if df_sample:
            arquivo_sample_txt = categoria_dir / f"{tabela_limpa}_SAMPLE.txt"
            arquivo_sample_csv = categoria_dir / f"{tabela_limpa}_SAMPLE.csv"
            
            if salvar_dataframe_como_texto(df_sample, arquivo_sample_txt):
                print(f"  ✓ Sample TXT salvo")
            if salvar_dataframe_como_csv(df_sample, arquivo_sample_csv):
                print(f"  ✓ Sample CSV salvo")
    
    print(f"\n{'='*80}")
    print(f"  ✅ Categoria {categoria} concluída!")
    print(f"{'='*80}")

print(f"\n\n🎉 TODAS AS CATEGORIAS RESTANTES PROCESSADAS!")

## 7. Criar Índice de Arquivos Gerados

In [ ]:
arquivo_indice = output_dir / "INDEX.md"

with open(arquivo_indice, 'w', encoding='utf-8') as f:
    f.write("# Data Schemas - Projeto ECD\n\n")
    f.write(f"Gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write("---\n\n")
    
    for categoria, tabelas in TABELAS.items():
        f.write(f"## {categoria}\n\n")
        
        for tabela in tabelas:
            tabela_limpa = tabela.replace('.', '_')
            f.write(f"### {tabela}\n\n")
            f.write(f"- **DESCRIBE**: `{categoria}/{tabela_limpa}_DESCRIBE.txt`\n")
            f.write(f"- **SAMPLE TXT**: `{categoria}/{tabela_limpa}_SAMPLE.txt`\n")
            f.write(f"- **SAMPLE CSV**: `{categoria}/{tabela_limpa}_SAMPLE.csv`\n\n")

print(f"✓ Índice criado: {arquivo_indice}")

## 8. Relatório Final

In [ ]:
import os

print("\n" + "="*80)
print("  RELATÓRIO FINAL - GERAÇÃO DE DATA SCHEMAS")
print("="*80)

# Contar arquivos gerados
total_arquivos = 0
arquivos_por_categoria = {}

for categoria in TABELAS.keys():
    categoria_dir = output_dir / categoria
    if categoria_dir.exists():
        arquivos = list(categoria_dir.glob("*"))
        arquivos_por_categoria[categoria] = len(arquivos)
        total_arquivos += len(arquivos)

print(f"\n📊 Estatísticas:")
print(f"  - Total de categorias processadas: {len(arquivos_por_categoria)}")
print(f"  - Total de arquivos gerados: {total_arquivos}")
print(f"  - Diretório de saída: {output_dir}")

print(f"\n📁 Arquivos por categoria:")
for categoria, num_arquivos in arquivos_por_categoria.items():
    print(f"  - {categoria}: {num_arquivos} arquivos")

print("\n" + "="*80)
print("  ✅ PROCESSAMENTO CONCLUÍDO COM SUCESSO!")
print("="*80)
print(f"\n📄 Consulte o índice completo em: {arquivo_indice}")
print(f"\n💡 Próximo passo: Revisar os arquivos gerados em {output_dir}")

## 9. Verificar Exemplos de Saída (Opcional)

In [ ]:
# Exemplo: Visualizar DESCRIBE de uma tabela
exemplo_tabela = "teste.ecd_balanco_patrimonial"

if verificar_tabela_existe(exemplo_tabela):
    print(f"\n📋 DESCRIBE FORMATTED: {exemplo_tabela}\n")
    df = spark.sql(f"DESCRIBE FORMATTED {exemplo_tabela}")
    df.show(50, truncate=False)
else:
    print(f"⚠️  Tabela {exemplo_tabela} não encontrada")

In [ ]:
# Exemplo: Visualizar SAMPLE de uma tabela
exemplo_tabela = "teste.ecd_balanco_patrimonial"

if verificar_tabela_existe(exemplo_tabela):
    print(f"\n📊 SAMPLE (10 linhas): {exemplo_tabela}\n")
    df = spark.sql(f"SELECT * FROM {exemplo_tabela} LIMIT 10")
    df.show(10, truncate=True)
else:
    print(f"⚠️  Tabela {exemplo_tabela} não encontrada")

---

## 📝 Notas Importantes

### Tabelas Particionadas

As seguintes tabelas são particionadas por `ano`:
- `teste.ecd_balanco_patrimonial`
- `teste.ecd_dre`
- `teste.ecd_indicadores_financeiros`

O DESCRIBE FORMATTED mostrará as informações de particionamento.

### Tabelas que Podem Não Existir

Algumas tabelas podem não existir se os notebooks correspondentes não foram executados:
- Tabelas ML (se modelo não foi treinado)
- Tabelas de validação (se análise não foi feita)

**O notebook pula automaticamente tabelas inexistentes.**

### Performance

- Tempo estimado: 10-20 minutos para processar todas as 52 tabelas
- Cada categoria pode ser executada independentemente
- Use LIMIT 10 para garantir performance

---